In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Model

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import sqlite3

import plotly.express as px

import plotly
import plotly.graph_objs as go

In [2]:
user = pd.read_csv('Data.csv')

print(user.shape)
user.head()

(1950, 7)


,ID user,Gender,Faculty,Major,Dormitory,Activity,Social
0,10001,หญิง,วิทยาศาตร์,เทคโนโลยีสารสนเทศ,At Home,"เล่นเกม,อ่านหนังสือ,เล่นดนตรี,ออกกำลังกาย,เล่น...",สามารถอยู่ร่วมกับเมทได้
1,10002,หญิง,เภสัชศาสตร์,บริบาลเภสัชกรรม,At home,"อ่านหนังสือ,เล่นโซเชียล,ทำความสะอาดห้อง,ทำการบ...",ชอบอยู่คนเดียว
2,10003,หญิง,มัณทณศิลป์,อินทีเรีย,At home,"สวดมนต์นั่งสมาธิ,เล่นโซเชียล,สร้างงานศิลปะ,ทำค...",ชอบอยู่คนเดียว
3,10004,หญิง,วิทยาศาตร์,วิทยาการข้อมูล,BPN,"เล่นเกม,เล่นโซเชียล,ทำความสะอาดห้อง,ดูหนัง,ทำก...",ชอบอยู่คนเดียว
4,10005,หญิง,วิศวะกรรมศาสตร์,ธุรกิจวิศวกรรม,DHAVA PLACE,"เล่นเกม,อ่านหนังสือ,สวดมนต์นั่งสมาธิ,เล่นโซเชี...",ชอบอยู่คนเดียว


In [3]:
rating = pd.read_csv('Rating.csv')

print(rating.shape)
rating.head()

(20144, 3)


,ID user,IDDor,rating
0,10001,1,1
1,10002,2,2
2,10003,3,3
3,10004,4,4
4,10005,5,5


In [4]:
dor = pd.read_csv('Dormitory.csv')

print(dor.shape)
dor.head()

(92, 3)


,IDDor,Dormitory,GenderDor
0,1,At Home,woman
1,2,BPN,man/woman
2,3,DHAVA PLACE,man/woman
3,4,I place,man/woman
4,5,Inroom,woman


In [5]:
connect = sqlite3.connect('dor_rec.sqlite3')

In [6]:
dor.to_sql("dor", connect, if_exists='fail')
user.to_sql("user", connect, if_exists='fail')
rating.to_sql("rating", connect, if_exists='fail')

F:\anaconda\lib\site-packages\pandas\core\generic.py:2653: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [7]:
%load_ext sql

In [8]:
%sql sqlite:///dor_rec.sqlite3

In [9]:
%sql SELECT * FROM rating LIMIT 5

 * sqlite:///dor_rec.sqlite3
Done.


index,ID user,IDDor,rating
0,10001,1,1
1,10002,2,2
2,10003,3,3
3,10004,4,4
4,10005,5,5


In [10]:
%sql SELECT * FROM dor LIMIT 5

 * sqlite:///dor_rec.sqlite3
Done.


index,IDDor,Dormitory,GenderDor
0,1,At Home,woman
1,2,BPN,man/woman
2,3,DHAVA PLACE,man/woman
3,4,I place,man/woman
4,5,Inroom,woman


In [11]:
%sql SELECT * FROM user LIMIT 5

 * sqlite:///dor_rec.sqlite3
Done.


index,ID user,Gender,Faculty,Major,Dormitory,Activity,Social
0,10001,หญิง,วิทยาศาตร์,เทคโนโลยีสารสนเทศ,At Home,"เล่นเกม,อ่านหนังสือ,เล่นดนตรี,ออกกำลังกาย,เล่นโซเชียล,ดูหนัง",สามารถอยู่ร่วมกับเมทได้
1,10002,หญิง,เภสัชศาสตร์,บริบาลเภสัชกรรม,At home,"อ่านหนังสือ,เล่นโซเชียล,ทำความสะอาดห้อง,ทำการบ้าน,นอนเล่น",ชอบอยู่คนเดียว
2,10003,หญิง,มัณทณศิลป์,อินทีเรีย,At home,"สวดมนต์นั่งสมาธิ,เล่นโซเชียล,สร้างงานศิลปะ,ทำความสะอาดห้อง,ทำการบ้าน,นอนเล่น",ชอบอยู่คนเดียว
3,10004,หญิง,วิทยาศาตร์,วิทยาการข้อมูล,BPN,"เล่นเกม,เล่นโซเชียล,ทำความสะอาดห้อง,ดูหนัง,ทำการบ้าน,นอนเล่น",ชอบอยู่คนเดียว
4,10005,หญิง,วิศวะกรรมศาสตร์,ธุรกิจวิศวกรรม,DHAVA PLACE,"เล่นเกม,อ่านหนังสือ,สวดมนต์นั่งสมาธิ,เล่นโซเชียล,สร้างงานศิลปะ,ทำอาหาร,ทำความสะอาดห้อง,ดูหนัง,ทำการบ้าน,นอน",ชอบอยู่คนเดียว


In [12]:
%%sql
CREATE TABLE rating_dor AS SELECT rating."ID user" AS UserID , rating.IDDor, rating."rating" AS DorRating, dor.Dormitory
FROM rating 
INNER JOIN dor
ON rating.IDDor = dor.IDDor

 * sqlite:///dor_rec.sqlite3
Done.


[]

In [13]:
%%sql
SELECT * FROM rating_dor LIMIT 5

 * sqlite:///dor_rec.sqlite3
Done.


UserID,IDDor,DorRating,Dormitory
10001,1,1,At Home
10002,2,2,BPN
10003,3,3,DHAVA PLACE
10004,4,4,I place
10005,5,5,Inroom


In [14]:
%%sql
SELECT count(*) FROM rating_dor

 * sqlite:///dor_rec.sqlite3
Done.


count(*)
20144


In [15]:
%%sql
CREATE TABLE rating_count AS SELECT "Dormitory", count(*) as RatingCountDor FROM rating_dor GROUP BY "Dormitory"

 * sqlite:///dor_rec.sqlite3
Done.


[]

In [16]:
%%sql
SELECT * from rating_count limit 5

 * sqlite:///dor_rec.sqlite3
Done.


Dormitory,RatingCountDor
At Home,219
BPN,229
DHAVA PLACE,234
I place,238
Inroom,213


In [17]:
%%sql
SELECT count(*) FROM rating_count

 * sqlite:///dor_rec.sqlite3
Done.


count(*)
92


In [18]:
rating_coun_threshold = 200

In [19]:
%%sql
CREATE TABLE rating_count_filter2 AS SELECT * FROM rating_count WHERE RatingCountDor >= :rating_coun_threshold

 * sqlite:///dor_rec.sqlite3
Done.


[]

In [20]:
%%sql
SELECT * FROM rating_count_filter2 LIMIT 5

 * sqlite:///dor_rec.sqlite3
Done.


Dormitory,RatingCountDor
At Home,219
BPN,229
DHAVA PLACE,234
I place,238
Inroom,213


In [21]:
%%sql
SELECT count(*) FROM rating_count_filter2

 * sqlite:///dor_rec.sqlite3
Done.


count(*)
84


In [22]:
%%sql
CREATE TABLE user_rating AS SELECT rating_dor.UserID, rating_dor.IDDor, rating_dor.DorRating, rating_dor.Dormitory, rating_count_filter2.RatingCountDor
FROM rating_count_filter2 
LEFT JOIN rating_dor
ON rating_count_filter2.Dormitory = rating_dor.Dormitory

 * sqlite:///dor_rec.sqlite3
Done.


[]

In [23]:
%%sql
SELECT * FROM user_rating LIMIT 5

 * sqlite:///dor_rec.sqlite3
Done.


UserID,IDDor,DorRating,Dormitory,RatingCountDor
10001,1,1,At Home,219
10005,1,2,At Home,219
10006,1,5,At Home,219
10016,1,5,At Home,219
10035,1,1,At Home,219


In [24]:
%%sql
SELECT count(*) FROM user_rating

 * sqlite:///dor_rec.sqlite3
Done.


count(*)
18607


In [25]:
%%sql
CREATE TABLE user_count AS SELECT UserID, count(*) as RatingCountDor FROM user_rating GROUP BY UserID


 * sqlite:///dor_rec.sqlite3
Done.


[]

In [26]:
%%sql
SELECT * FROM user_count LIMIT 5

 * sqlite:///dor_rec.sqlite3
Done.


UserID,RatingCountDor
10001,8
10002,10
10003,10
10004,8
10005,13


In [27]:
%%sql
SELECT count(*) FROM user_count

 * sqlite:///dor_rec.sqlite3
Done.


count(*)
1950


In [28]:
user_count_filter_threshold = 10

In [29]:
%%sql
CREATE TABLE user_count_filter2 AS SELECT * FROM user_count WHERE RatingCountDor >= :user_count_filter_threshold

 * sqlite:///dor_rec.sqlite3
Done.


[]

In [30]:
%%sql
SELECT * FROM user_count_filter2 LIMIT 5

 * sqlite:///dor_rec.sqlite3
Done.


UserID,RatingCountDor
10002,10
10003,10
10005,13
10009,10
10010,10


In [31]:
%%sql
SELECT count(*) FROM user_count_filter2

 * sqlite:///dor_rec.sqlite3
Done.


count(*)
942


In [32]:
%%sql
CREATE TABLE combined AS SELECT user_rating.Dormitory, user_rating.RatingCountDor, user_rating.UserID, user_rating.IDDor, user_rating.DorRating, user_count_filter2.RatingCountDor
FROM user_rating
INNER JOIN user_count_filter2
ON user_rating.UserID = user_count_filter2.UserID

 * sqlite:///dor_rec.sqlite3
Done.


[]

In [33]:
%%sql
SELECT * FROM combined LIMIT 5

 * sqlite:///dor_rec.sqlite3
Done.


Dormitory,RatingCountDor,UserID,IDDor,DorRating,RatingCountDor:1
At Home,219,10005,1,2,13
At Home,219,10016,1,5,13
At Home,219,10035,1,1,12
At Home,219,10035,1,5,12
At Home,219,10037,1,1,11


In [34]:
%%sql
select count(distinct Dormitory) AS NumberOfUniqueDormitory
FROM combined

 * sqlite:///dor_rec.sqlite3
Done.


NumberOfUniqueDormitory
84


In [35]:
%%sql
select count(distinct UserID) AS NumberOfUniqueUser
FROM combined

 * sqlite:///dor_rec.sqlite3
Done.


NumberOfUniqueUser
942


In [36]:
combined = %sql SELECT * FROM combined

 * sqlite:///dor_rec.sqlite3
Done.


In [37]:
combined_df = pd.DataFrame(data = combined, columns = combined.field_names)

combined_df.head()

,Dormitory,RatingCountDor,UserID,IDDor,DorRating,RatingCountDor:1
0,At Home,219,10005,1,2,13
1,At Home,219,10016,1,5,13
2,At Home,219,10035,1,1,12
3,At Home,219,10035,1,5,12
4,At Home,219,10037,1,1,11


In [38]:
combined_df['DorRating'] = combined_df['DorRating'].values.astype(float)

print(combined_df.shape)
combined_df.head()

(11370, 6)


,Dormitory,RatingCountDor,UserID,IDDor,DorRating,RatingCountDor:1
0,At Home,219,10005,1,2.0,13
1,At Home,219,10016,1,5.0,13
2,At Home,219,10035,1,1.0,12
3,At Home,219,10035,1,5.0,12
4,At Home,219,10037,1,1.0,11


In [39]:
combined_df = combined_df.drop_duplicates(['UserID', 'Dormitory'])

print(combined_df.shape)
combined_df.head()

(10659, 6)


,Dormitory,RatingCountDor,UserID,IDDor,DorRating,RatingCountDor:1
0,At Home,219,10005,1,2.0,13
1,At Home,219,10016,1,5.0,13
2,At Home,219,10035,1,1.0,12
4,At Home,219,10037,1,1.0,11
5,At Home,219,10056,1,5.0,11


In [40]:
user_dor_matrix = combined_df.pivot(index='UserID', columns='Dormitory', values='DorRating')

print(user_dor_matrix.shape)
user_dor_matrix.head()

(942, 84)


Dormitory,At Home,BPN,DHAVA PLACE,I place,Inroom,JK place,KP Home,L20 Residents,PBN แมนชั่น,PJ Smile Place,...,ห้องเช่า,อนงค์,เจอาร์,เดอะพฤกษ์,เบญจรงค์,เรือนขวัญ,เรือนปฐมบุรี,เวศย์วรุตม์เพลส,โครงการหมู่บ้านพฤกษาพรรณ กระดีเดียว1,ไดพักทิวาลัย
UserID,,,,,,,,,,,,,,,,,,,,,
10002,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,2.0,NaN,5.0,NaN,NaN
10003,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN
10005,2.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
10009,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,4.0
10010,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
user_dor_matrix.fillna(0, inplace=True)

user_dor_matrix.head()

Dormitory,At Home,BPN,DHAVA PLACE,I place,Inroom,JK place,KP Home,L20 Residents,PBN แมนชั่น,PJ Smile Place,...,ห้องเช่า,อนงค์,เจอาร์,เดอะพฤกษ์,เบญจรงค์,เรือนขวัญ,เรือนปฐมบุรี,เวศย์วรุตม์เพลส,โครงการหมู่บ้านพฤกษาพรรณ กระดีเดียว1,ไดพักทิวาลัย
UserID,,,,,,,,,,,,,,,,,,,,,
10002,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0
10003,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
10005,2.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
10009,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0
10010,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
users = user_dor_matrix.index.tolist()

users[0:10]

[10002, 10003, 10005, 10009, 10010, 10012, 10014, 10016, 10020, 10022]

In [43]:
dors = user_dor_matrix.columns.tolist()

dors[0:10]

['At Home',
 'BPN',
 'DHAVA PLACE ',
 'I place',
 'Inroom',
 'JK place',
 'KP Home',
 'L20 Residents ',
 'PBN แมนชั่น',
 'PJ Smile Place']

In [44]:
user_dor_matrix = user_dor_matrix.to_numpy()

user_dor_matrix.shape

(942, 84)

In [45]:
X_train, X_test, y_train, y_test = train_test_split(user_dor_matrix, user_dor_matrix, test_size=0.2, random_state=1)

In [46]:
t = MinMaxScaler()

t.fit(user_dor_matrix)
X_train = t.transform(X_train)
X_test = t.transform(X_test)

In [47]:
print(X_train.min(), X_train.max())
print(X_test.min(), X_test.max())

0.0 1.0
0.0 1.0


In [48]:
num_input = len(dors)

inp = Input((num_input))
e = Dense(10)(inp)
e = BatchNormalization()(e)
e = LeakyReLU()(e)

n_bottleneck = 5
bottleneck = Dense(n_bottleneck)(e)

In [49]:
d = Dense(10)(bottleneck)
d = BatchNormalization()(d)
d = LeakyReLU()(d)

decoded = Dense(num_input, activation='sigmoid')(d)
                
ae = Model(inp, decoded)
ae.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 84)]              0         
_________________________________________________________________
dense (Dense)                (None, 10)                850       
_________________________________________________________________
batch_normalization (BatchNo (None, 10)                40        
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_2 (Dense)              (None, 10)                60        
_________________________________________________________________
batch_normalization_1 (Batch (None, 10)               

In [50]:
ae.compile(loss='mse', optimizer='adam')

In [51]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [52]:
history = ae.fit(X_train, X_train, epochs=300, verbose=1, batch_size=32, validation_data=(X_test, X_test), callbacks = [checkpoint], shuffle= True)

Epoch 1/300
19/24 [======================>.......] - ETA: 0s - loss: 0.2330
Epoch 00001: val_loss improved from inf to 0.22244, saving model to model.h5
24/24 [==============================] - 0s 16ms/step - loss: 0.2311 - val_loss: 0.2224
Epoch 2/300
 1/24 [>.............................] - ETA: 0s - loss: 0.2207
Epoch 00002: val_loss improved from 0.22244 to 0.21564, saving model to model.h5
24/24 [==============================] - 0s 4ms/step - loss: 0.2138 - val_loss: 0.2156
Epoch 3/300
 1/24 [>.............................] - ETA: 0s - loss: 0.2044
Epoch 00003: val_loss improved from 0.21564 to 0.20790, saving model to model.h5
24/24 [==============================] - 0s 4ms/step - loss: 0.1971 - val_loss: 0.2079
Epoch 4/300
 1/24 [>.............................] - ETA: 0s - loss: 0.1867
Epoch 00004: val_loss improved from 0.20790 to 0.19788, saving model to model.h5
24/24 [==============================] - 0s 4ms/step - loss: 0.1798 - val_loss: 0.1979
Epoch 5/300
 1/24 [>.......

24/24 [==============================] - 0s 4ms/step - loss: 0.0498 - val_loss: 0.0507
Epoch 35/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0500
Epoch 00035: val_loss improved from 0.05071 to 0.05066, saving model to model.h5
24/24 [==============================] - 0s 4ms/step - loss: 0.0497 - val_loss: 0.0507
Epoch 36/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0480
Epoch 00036: val_loss improved from 0.05066 to 0.05056, saving model to model.h5
24/24 [==============================] - 0s 4ms/step - loss: 0.0496 - val_loss: 0.0506
Epoch 37/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0504
Epoch 00037: val_loss improved from 0.05056 to 0.05042, saving model to model.h5
24/24 [==============================] - 0s 4ms/step - loss: 0.0495 - val_loss: 0.0504
Epoch 38/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0468
Epoch 00038: val_loss improved from 0.05042 to 0.05034, saving model to model.h5
24/24 [=============

24/24 [==============================] - 0s 4ms/step - loss: 0.0475 - val_loss: 0.0488
Epoch 69/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0475
Epoch 00069: val_loss improved from 0.04876 to 0.04873, saving model to model.h5
24/24 [==============================] - 0s 4ms/step - loss: 0.0475 - val_loss: 0.0487
Epoch 70/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0465
Epoch 00070: val_loss improved from 0.04873 to 0.04871, saving model to model.h5
24/24 [==============================] - 0s 3ms/step - loss: 0.0474 - val_loss: 0.0487
Epoch 71/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0463
Epoch 00071: val_loss improved from 0.04871 to 0.04863, saving model to model.h5
24/24 [==============================] - 0s 3ms/step - loss: 0.0474 - val_loss: 0.0486
Epoch 72/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0484
Epoch 00072: val_loss improved from 0.04863 to 0.04860, saving model to model.h5
24/24 [=============

 1/24 [>.............................] - ETA: 0s - loss: 0.0484
Epoch 00103: val_loss improved from 0.04777 to 0.04773, saving model to model.h5
24/24 [==============================] - 0s 4ms/step - loss: 0.0461 - val_loss: 0.0477
Epoch 104/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0499
Epoch 00104: val_loss improved from 0.04773 to 0.04770, saving model to model.h5
24/24 [==============================] - 0s 3ms/step - loss: 0.0461 - val_loss: 0.0477
Epoch 105/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0423
Epoch 00105: val_loss improved from 0.04770 to 0.04768, saving model to model.h5
24/24 [==============================] - 0s 4ms/step - loss: 0.0461 - val_loss: 0.0477
Epoch 106/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0426
Epoch 00106: val_loss improved from 0.04768 to 0.04768, saving model to model.h5
24/24 [==============================] - 0s 4ms/step - loss: 0.0460 - val_loss: 0.0477
Epoch 107/300
 1/24 [>........

Epoch 138/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0497
Epoch 00138: val_loss improved from 0.04706 to 0.04704, saving model to model.h5
24/24 [==============================] - 0s 3ms/step - loss: 0.0449 - val_loss: 0.0470
Epoch 139/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0451
Epoch 00139: val_loss improved from 0.04704 to 0.04704, saving model to model.h5
24/24 [==============================] - 0s 4ms/step - loss: 0.0448 - val_loss: 0.0470
Epoch 140/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0485
Epoch 00140: val_loss improved from 0.04704 to 0.04702, saving model to model.h5
24/24 [==============================] - 0s 4ms/step - loss: 0.0447 - val_loss: 0.0470
Epoch 141/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0423
Epoch 00141: val_loss did not improve from 0.04702
24/24 [==============================] - 0s 2ms/step - loss: 0.0448 - val_loss: 0.0470
Epoch 142/300
 1/24 [>........................

Epoch 173/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0446
Epoch 00173: val_loss improved from 0.04667 to 0.04662, saving model to model.h5
24/24 [==============================] - 0s 4ms/step - loss: 0.0440 - val_loss: 0.0466
Epoch 174/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0431
Epoch 00174: val_loss improved from 0.04662 to 0.04659, saving model to model.h5
24/24 [==============================] - 0s 4ms/step - loss: 0.0440 - val_loss: 0.0466
Epoch 175/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0378
Epoch 00175: val_loss did not improve from 0.04659
24/24 [==============================] - 0s 2ms/step - loss: 0.0439 - val_loss: 0.0466
Epoch 176/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0461
Epoch 00176: val_loss did not improve from 0.04659
24/24 [==============================] - 0s 2ms/step - loss: 0.0440 - val_loss: 0.0466
Epoch 177/300
 1/24 [>.............................] - ETA: 0s - loss: 0.045

Epoch 209/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0465
Epoch 00209: val_loss did not improve from 0.04634
24/24 [==============================] - 0s 2ms/step - loss: 0.0435 - val_loss: 0.0464
Epoch 210/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0461
Epoch 00210: val_loss did not improve from 0.04634
24/24 [==============================] - 0s 2ms/step - loss: 0.0435 - val_loss: 0.0464
Epoch 211/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0425
Epoch 00211: val_loss did not improve from 0.04634
24/24 [==============================] - 0s 2ms/step - loss: 0.0433 - val_loss: 0.0464
Epoch 212/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0422
Epoch 00212: val_loss did not improve from 0.04634
24/24 [==============================] - 0s 2ms/step - loss: 0.0435 - val_loss: 0.0464
Epoch 213/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0447
Epoch 00213: val_loss did not improve from 0.04634
24/24 [

Epoch 247/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0456
Epoch 00247: val_loss did not improve from 0.04631
24/24 [==============================] - 0s 2ms/step - loss: 0.0430 - val_loss: 0.0463
Epoch 248/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0433
Epoch 00248: val_loss did not improve from 0.04631
24/24 [==============================] - 0s 2ms/step - loss: 0.0430 - val_loss: 0.0463
Epoch 249/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0390
Epoch 00249: val_loss did not improve from 0.04631
24/24 [==============================] - 0s 2ms/step - loss: 0.0430 - val_loss: 0.0463
Epoch 250/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0473
Epoch 00250: val_loss did not improve from 0.04631
24/24 [==============================] - 0s 2ms/step - loss: 0.0430 - val_loss: 0.0464
Epoch 251/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0437
Epoch 00251: val_loss did not improve from 0.04631
24/24 [

 1/24 [>.............................] - ETA: 0s - loss: 0.0424
Epoch 00284: val_loss did not improve from 0.04622
24/24 [==============================] - 0s 2ms/step - loss: 0.0428 - val_loss: 0.0463
Epoch 285/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0432
Epoch 00285: val_loss did not improve from 0.04622
24/24 [==============================] - 0s 2ms/step - loss: 0.0429 - val_loss: 0.0463
Epoch 286/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0423
Epoch 00286: val_loss did not improve from 0.04622
24/24 [==============================] - 0s 2ms/step - loss: 0.0428 - val_loss: 0.0463
Epoch 287/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0412
Epoch 00287: val_loss did not improve from 0.04622
24/24 [==============================] - 0s 2ms/step - loss: 0.0427 - val_loss: 0.0463
Epoch 288/300
 1/24 [>.............................] - ETA: 0s - loss: 0.0438
Epoch 00288: val_loss did not improve from 0.04622
24/24 [==============

In [54]:
plotly.offline.init_notebook_mode(connected=True)

h1 = go.Scatter(y=history.history['loss'], 
                    mode="lines", line=dict(
                    width=2,
                    color='blue'),
                    name="loss"
                   )
h2 = go.Scatter(y=history.history['val_loss'], 
                    mode="lines", line=dict(
                    width=2,
                    color='red'),
                    name="val_loss"
                   )

data = [h1,h2]
layout1 = go.Layout(title='Loss',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
plotly.offline.iplot(fig1, filename="Autoencoder")

In [55]:
predict_model = load_model(filename) 
predict_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 84)]              0         
_________________________________________________________________
dense (Dense)                (None, 10)                850       
_________________________________________________________________
batch_normalization (BatchNo (None, 10)                40        
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_2 (Dense)              (None, 10)                60        
_________________________________________________________________
batch_normalization_1 (Batch (None, 10)               

In [56]:
print(user_dor_matrix)

preds = predict_model(user_dor_matrix)

[[0. 1. 0. ... 5. 0. 0.]
 [0. 0. 3. ... 0. 0. 0.]
 [2. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [57]:
preds = preds.numpy()

pred_data = pd.DataFrame(preds)
pred_data.head()

,0,1,2,3,4,5,6,7,8,9,...,74,75,76,77,78,79,80,81,82,83
0,3.744086e-18,1.160394e-05,0.011568,0.000061,0.000537,0.000436,5.688142e-05,0.001023,3.908634e-11,0.000347,...,0.000038,1.064379e-19,0.000800,0.051507,0.006390,0.033475,0.006183,0.001872,1.836419e-04,6.358583e-08
1,9.099545e-01,1.606405e-03,0.034956,0.020765,0.015756,0.050307,8.755624e-04,0.061368,3.877076e-06,0.061161,...,0.000160,8.802128e-01,0.018396,0.009652,0.004724,0.015729,0.066605,0.006465,7.655698e-02,1.017313e-05
2,9.887159e-03,1.797736e-03,0.014229,0.121976,0.093333,0.060242,4.136026e-02,0.088723,1.483696e-07,0.103042,...,0.000008,1.198071e-01,0.145516,0.023071,0.057592,0.044375,0.126765,0.029126,7.754916e-02,5.305335e-05
3,1.142374e-06,1.378953e-04,0.002847,0.000035,0.000196,0.006262,3.761712e-24,0.000014,1.000000e+00,0.000277,...,0.994249,9.671749e-07,0.000354,0.002376,0.000127,0.000058,0.000001,0.000066,1.319306e-05,7.498395e-01
4,2.062422e-22,4.307642e-14,0.011866,0.025414,0.000008,0.000799,1.316506e-14,0.000118,6.876323e-10,0.000122,...,0.456699,7.251455e-13,0.019296,0.004724,0.000710,0.000489,0.002067,0.005664,1.411719e-07,7.040125e-11


In [58]:
pred_data = pred_data.stack().reset_index()
pred_data.head()

,level_0,level_1,0
0,0,0,3.744086e-18
1,0,1,1.160394e-05
2,0,2,1.156756e-02
3,0,3,6.106846e-05
4,0,4,5.371869e-04


In [59]:
pred_data.columns = ['UserID', 'Dormitory', 'DorRating']

print(pred_data.shape)
pred_data.head()

(79128, 3)


,UserID,Dormitory,DorRating
0,0,0,3.744086e-18
1,0,1,1.160394e-05
2,0,2,1.156756e-02
3,0,3,6.106846e-05
4,0,4,5.371869e-04


In [60]:
pred_data['UserID'] = pred_data['UserID'].map(lambda value: users[value])

pred_data.head()

,UserID,Dormitory,DorRating
0,10002,0,3.744086e-18
1,10002,1,1.160394e-05
2,10002,2,1.156756e-02
3,10002,3,6.106846e-05
4,10002,4,5.371869e-04


In [61]:
pred_data['Dormitory'] = pred_data['Dormitory'].map(lambda value: dors[value])

pred_data.head()

,UserID,Dormitory,DorRating
0,10002,At Home,3.744086e-18
1,10002,BPN,1.160394e-05
2,10002,DHAVA PLACE,1.156756e-02
3,10002,I place,6.106846e-05
4,10002,Inroom,5.371869e-04


In [62]:
pred_data.to_sql("pred_data", connect, if_exists='fail')

In [67]:
%%sql
SELECT * 
FROM pred_data 
WHERE UserID=10002
ORDER BY DorRating DESC
LIMIT 10

 * sqlite:///dor_rec.sqlite3
Done.


index,UserID,Dormitory,DorRating
58,10002,ระพีพรรณ,0.9999997615814209
34,10002,ทิพย์ชยพล,0.999030590057373
33,10002,ทับเเก้ว,0.6764224171638489
40,10002,นาราชา เรสซิเดนท์,0.6287815570831299
19,10002,กินนอน,0.4559347927570343
54,10002,บ้านไทย,0.273189902305603
62,10002,วานิลลา,0.2301681637763977
12,10002,The loft condo,0.14636507630348206
67,10002,สตรีบ้านอุบลวรรณ,0.12585550546646118
69,10002,สนามจันทร์ คอร์ส,0.10855311155319214


In [68]:
iduser2=10002

In [69]:
%%sql
SELECT * 
FROM pred_data 
WHERE UserID == {iduser2}
ORDER BY DorRating DESC
LIMIT 10

 * sqlite:///dor_rec.sqlite3
Done.


index,UserID,Dormitory,DorRating
58,10002,ระพีพรรณ,0.9999997615814209
34,10002,ทิพย์ชยพล,0.999030590057373
33,10002,ทับเเก้ว,0.6764224171638489
40,10002,นาราชา เรสซิเดนท์,0.6287815570831299
19,10002,กินนอน,0.4559347927570343
54,10002,บ้านไทย,0.273189902305603
62,10002,วานิลลา,0.2301681637763977
12,10002,The loft condo,0.14636507630348206
67,10002,สตรีบ้านอุบลวรรณ,0.12585550546646118
69,10002,สนามจันทร์ คอร์ส,0.10855311155319214


In [80]:
ipython nbconvert — to script abc.ipynb

SyntaxError: invalid syntax (<ipython-input-80-bd8be5e7ef5f>, line 1)